In [ ]:
# Standard library imports
import os
import warnings
import pickle as pk

# Data manipulation and numerical computing
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_rel

# Computer vision and image processing
import cv2

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning and neural networks
import tensorflow
from tensorflow import keras, convert_to_tensor
from tensorflow.keras import preprocessing
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

# Keras model components
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten
from keras.applications.vgg16 import VGG16, preprocess_input

# Machine learning models and utilities
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_val_predict

# Model evaluation metrics
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix

# Data balancing techniques
from imblearn.over_sampling import BorderlineSMOTE

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

reading image sample from the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set up the feature extractor

In [ ]:
# Generate list of absolute file paths for meningioma tumor images
# This collects all images from the meningioma directory and converts to absolute paths
meningioma = [os.path.abspath(os.path.join('/content/drive/MyDrive/f data/3class/meningioma', p))
              for p in os.listdir('/content/drive/MyDrive/f data/3class/meningioma')]

# Generate list of absolute file paths for glioma tumor images
# This collects all images from the glioma directory and converts to absolute paths
glioma = [os.path.abspath(os.path.join('/content/drive/MyDrive/f data/3class/glioma', p))
          for p in os.listdir('/content/drive/MyDrive/f data/3class/glioma')]

# Generate list of absolute file paths for pituitary tumor images
# This collects all images from the pituitary directory and converts to absolute paths
pituitary = [os.path.abspath(os.path.join('/content/drive/MyDrive/f data/3class/pituitary', p))
             for p in os.listdir('/content/drive/MyDrive/f data/3class/pituitary')]

In [ ]:
# Load pre-trained DenseNet121 model without the top classification layers
# This allows us to use the model as a feature extractor
# include_top=False removes the final fully connected layers
# input_shape=(224, 224, 3) specifies the expected image dimensions
model = DenseNet121(include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model to prevent training
# This preserves the pre-trained ImageNet weights and speeds up training
# Since we're using it as a feature extractor, we don't want to update these weights
for layer in model.layers:
    layer.trainable = False

# Add a Flatten layer to convert the 3D feature maps (7x7x1024) into a 1D feature vector
# This transforms the convolutional features into a format suitable for classifiers like SVM/KNN
# model.layers[-1].output gets the output from the last layer of DenseNet121
output = Flatten()(model.layers[-1].output)

# Create a new Model that takes the same input but outputs the flattened features
# This creates our final feature extractor model
model = Model(inputs=model.inputs, outputs=output)

# Display the model architecture to verify the structure
# Shows the layer types, output shapes, and number of parameters
model.summary()

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,037,504 (26.85 MB)

In [ ]:
def get_features(img_path):
    """
    Extracts deep features from an MRI image using a pre-trained model.

    Args:
        img_path (str): Path to the input MRI image file

    Returns:
        numpy.ndarray: Flattened feature vector of dtype float16
    """
    # Load and resize image to match model's expected input dimensions
    img = load_img(img_path, target_size=(224, 224, 3))

    # Convert image to array, preprocess for the specific model (e.g., VGG16/DenseNet121),
    # and extract features using the pre-trained model
    # - np.expand_dims: Adds batch dimension (converts from 224x224x3 to 1x224x224x3)
    # - preprocess_input: Applies model-specific preprocessing (e.g., channel-wise centering)
    # - model.predict: Runs forward pass to get features (verbose=0 for silent operation)
    flatten = model.predict(preprocess_input(np.expand_dims(img_to_array(img), axis=0)), verbose=0)

    # Extract features from batch output and convert to numpy array
    fd = np.array(flatten[0])

    # Convert to float16 to reduce memory usage while maintaining sufficient precision
    fd = fd.astype('float16')

    return fd

In [ ]:
# Initialize empty lists to store extracted features and corresponding labels
featuresb1 = []
labels = []

# Define the feature vector length (n=50176 for flattened DenseNet121 features)
n = 50176
i = 0  # Counter to handle first sample initialization

# Process glioma tumor images
for image_path in glioma:
    # Extract features and reshape to 1D vector of length n
    feature_vector = np.reshape(get_features(image_path), (1, n))

    # For the first sample, initialize the features array
    if i == 0:
        featuresb1 = feature_vector
        print(f"Initialized feature array shape: {featuresb1.shape}")
    else:
        # Append subsequent samples to the existing array
        featuresb1 = np.append(featuresb1, feature_vector, axis=0)

    # Add corresponding label
    labels.append('glioma')
    i += 1

# Process meningioma tumor images
for image_path in meningioma:
    # Extract features and append to existing array
    feature_vector = np.reshape(get_features(image_path), (1, n))
    featuresb1 = np.append(featuresb1, feature_vector, axis=0)

    # Add corresponding label
    labels.append('meningioma')

# Process pituitary tumor images
for image_path in pituitary:
    # Extract features and append to existing array
    feature_vector = np.reshape(get_features(image_path), (1, n))
    featuresb1 = np.append(featuresb1, feature_vector, axis=0)

    # Add corresponding label
    labels.append('pituitary')

Initialized feature array shape: (1, 50176)


In [ ]:
featuresf =  featuresb1
labelsf = labels

In [ ]:
labelsf = [int(0) if x=='meningioma' else x for x in labelsf]
labelsf= [int(1) if x=='glioma' else x for x in labelsf]
labelsf= [int(2) if x=='pituitary' else x for x in labelsf]

In [ ]:
# Convert to numpy arrays if they aren't already
featuresf_array = np.array(featuresb1)
labelsf_array = np.array(labelsf)

# Initialize classification models with fixed random state for reproducibility
svc = SVC(C=10, random_state=42)
knn = KNeighborsClassifier(n_neighbors=1)
# Initialize 5-fold stratified cross-validation to maintain class distribution
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store accuracy scores for each fold
svc_scores = []
knn_scores = []

print("Starting 5-Fold Cross Validation with DenseNet121 features...")
print("Configuration: PCA applied after split (1000 components), no SMOTE")

for fold, (train_idx, test_idx) in enumerate(kf.split(featuresf_array, labelsf_array), 1):
    print(f"\n--- Processing Fold {fold}/5 ---")

    # Split data into training and testing sets for current fold
    X_train, X_test = featuresf_array[train_idx], featuresf_array[test_idx]
    y_train, y_test = labelsf_array[train_idx], labelsf_array[test_idx]

    # Apply PCA for dimensionality reduction (50176 -> 1000 features)
    # Important: Fit PCA ONLY on training data to avoid data leakage
    pca = PCA(n_components=1000, random_state=42)
    X_train_pca = pca.fit_transform(X_train)  # Fit and transform training data
    X_test_pca = pca.transform(X_test)        # Transform test data using same PCA model

    # Train and evaluate Support Vector Classifier
    svc.fit(X_train_pca, y_train)
    y_pred_svc = svc.predict(X_test_pca)
    svc_score = accuracy_score(y_test, y_pred_svc) * 100  # Convert to percentage
    svc_scores.append(svc_score)

    # Train and evaluate K-Nearest Neighbors classifier
    knn.fit(X_train_pca, y_train)
    y_pred_knn = knn.predict(X_test_pca)
    knn_score = accuracy_score(y_test, y_pred_knn) * 100  # Convert to percentage
    knn_scores.append(knn_score)

    print(f"Fold {fold} Results:")
    print(f"  SVC Accuracy: {svc_score:.2f}%")
    print(f"  KNN Accuracy: {knn_score:.2f}%")

# Display comprehensive final results
print("\n" + "="*50)
print("FINAL 5-FOLD CROSS VALIDATION RESULTS")
print("Feature Source: DenseNet121 (50176 features reduced to 1000 via PCA)")
print("="*50)
print(f"SVC Average Accuracy: {np.mean(svc_scores):.2f}% (±{np.std(svc_scores):.2f}%)")
print(f"KNN Average Accuracy: {np.mean(knn_scores):.2f}% (±{np.std(knn_scores):.2f}%)")
print(f"Individual SVC scores: {[f'{s:.2f}%' for s in svc_scores]}")
print(f"Individual KNN scores: {[f'{s:.2f}%' for s in knn_scores]}")
print("="*50)

Starting 5-Fold Cross Validation with DenseNet121 features...
Configuration: PCA applied after split (1000 components), no SMOTE

--- Processing Fold 1/5 ---
Fold 1 Results:
  SVC Accuracy: 94.94%
  KNN Accuracy: 95.60%

--- Processing Fold 2/5 ---
Fold 2 Results:
  SVC Accuracy: 96.25%
  KNN Accuracy: 95.76%

--- Processing Fold 3/5 ---
Fold 3 Results:
  SVC Accuracy: 97.72%
  KNN Accuracy: 96.08%

--- Processing Fold 4/5 ---
Fold 4 Results:
  SVC Accuracy: 96.74%
  KNN Accuracy: 96.57%

--- Processing Fold 5/5 ---
Fold 5 Results:
  SVC Accuracy: 95.92%
  KNN Accuracy: 95.92%

FINAL 5-FOLD CROSS VALIDATION RESULTS
Feature Source: DenseNet121 (50176 features reduced to 1000 via PCA)
SVC Average Accuracy: 96.31% (±0.92%)
KNN Average Accuracy: 95.99% (±0.34%)
Individual SVC scores: ['94.94%', '96.25%', '97.72%', '96.74%', '95.92%']
Individual KNN scores: ['95.60%', '95.76%', '96.08%', '96.57%', '95.92%']


In [ ]:
# Load pre-trained VGG16 model as a feature extractor (without classification layers)
model = VGG16(include_top=False, input_shape=(224, 224, 3))

# Freeze all layers to preserve pre-trained weights and speed up processing
for layer in model.layers:
    layer.trainable = False

# Convert the final convolutional layer output to 1D feature vector
output = Flatten()(model.layers[-1].output)

# Create the final feature extraction model
model = Model(inputs=model.inputs, outputs=output)

# Display model architecture for verification
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Initialize feature array and labels list for VGG16 features
# VGG16 produces 512x7x7 = 25088 features after flattening
featuresb2 = []
labels = []
n = 25088  # Feature vector size for VGG16

# Counter for handling first sample initialization
i=0

# Extract VGG16 features from glioma images
for image_path in glioma:
    # Get features and reshape to 1D vector
    feature_vector = np.reshape(get_features(image_path), (1, n))

    # Initialize array with first sample or append subsequent ones
    if i == 0:
        featuresb2 = feature_vector
        print(f"VGG16 feature array shape: {featuresb2.shape}")
    else:
        featuresb2 = np.append(featuresb2, feature_vector, axis=0)

    labels.append('glioma')
    i+=1

# Extract VGG16 features from meningioma images
for image_path in meningioma:
    feature_vector = np.reshape(get_features(image_path), (1, n))
    featuresb2 = np.append(featuresb2, feature_vector, axis=0)
    labels.append('meningioma')

# Extract VGG16 features from pituitary images
for image_path in pituitary:
    feature_vector = np.reshape(get_features(image_path), (1, n))
    featuresb2 = np.append(featuresb2, feature_vector, axis=0)
    labels.append('pituitary')

VGG16 feature array shape: (1, 25088)


In [ ]:
featuresf =  featuresb2
labelsf = labels

In [ ]:
labelsf = [int(0) if x=='meningioma' else x for x in labelsf]
labelsf= [int(1) if x=='glioma' else x for x in labelsf]
labelsf= [int(2) if x=='pituitary' else x for x in labelsf]

In [ ]:
# Convert to numpy arrays if they aren't already
featuresf_array = np.array(featuresb2)
labelsf_array = np.array(labelsf)

# Initialize classification models with fixed random state for reproducibility
svc = SVC(C=10, random_state=42)
knn = KNeighborsClassifier(n_neighbors=1)
# Initialize 5-fold stratified cross-validation to maintain class distribution
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store accuracy scores for each fold
svc_scores = []
knn_scores = []

print("Starting 5-Fold Cross Validation with VGG16 features...")
print("Configuration: PCA applied after split (1000 components), no SMOTE")

for fold, (train_idx, test_idx) in enumerate(kf.split(featuresf_array, labelsf_array), 1):
    print(f"\n--- Processing Fold {fold}/5 ---")

    # Split data into training and testing sets for current fold
    X_train, X_test = featuresf_array[train_idx], featuresf_array[test_idx]
    y_train, y_test = labelsf_array[train_idx], labelsf_array[test_idx]

    # Apply PCA for dimensionality reduction (25088 -> 1000 features)
    # Important: Fit PCA ONLY on training data to avoid data leakage
    pca = PCA(n_components=1000, random_state=42)
    X_train_pca = pca.fit_transform(X_train)  # Fit and transform training data
    X_test_pca = pca.transform(X_test)        # Transform test data using same PCA model

    # Train and evaluate Support Vector Classifier
    svc.fit(X_train_pca, y_train)
    y_pred_svc = svc.predict(X_test_pca)
    svc_score = accuracy_score(y_test, y_pred_svc) * 100  # Convert to percentage
    svc_scores.append(svc_score)

    # Train and evaluate K-Nearest Neighbors classifier
    knn.fit(X_train_pca, y_train)
    y_pred_knn = knn.predict(X_test_pca)
    knn_score = accuracy_score(y_test, y_pred_knn) * 100  # Convert to percentage
    knn_scores.append(knn_score)

    print(f"Fold {fold} Results:")
    print(f"  SVC Accuracy: {svc_score:.2f}%")
    print(f"  KNN Accuracy: {knn_score:.2f}%")

# Display comprehensive final results
print("\n" + "="*50)
print("FINAL 5-FOLD CROSS VALIDATION RESULTS")
print("Feature Source: VGG16 (25088 features reduced to 1000 via PCA)")
print("="*50)
print(f"SVC Average Accuracy: {np.mean(svc_scores):.2f}% (±{np.std(svc_scores):.2f}%)")
print(f"KNN Average Accuracy: {np.mean(knn_scores):.2f}% (±{np.std(knn_scores):.2f}%)")
print(f"Individual SVC scores: {[f'{s:.2f}%' for s in svc_scores]}")
print(f"Individual KNN scores: {[f'{s:.2f}%' for s in knn_scores]}")
print("="*50)

Starting 5-Fold Cross Validation with VGG16 features...
Configuration: PCA applied after split (1000 components), no SMOTE

--- Processing Fold 1/5 ---
Fold 1 Results:
  SVC Accuracy: 94.78%
  KNN Accuracy: 93.96%

--- Processing Fold 2/5 ---
Fold 2 Results:
  SVC Accuracy: 95.27%
  KNN Accuracy: 94.62%

--- Processing Fold 3/5 ---
Fold 3 Results:
  SVC Accuracy: 95.76%
  KNN Accuracy: 94.62%

--- Processing Fold 4/5 ---
Fold 4 Results:
  SVC Accuracy: 95.60%
  KNN Accuracy: 96.08%

--- Processing Fold 5/5 ---
Fold 5 Results:
  SVC Accuracy: 97.22%
  KNN Accuracy: 95.26%

FINAL 5-FOLD CROSS VALIDATION RESULTS
Feature Source: VGG16 (25088 features reduced to 1000 via PCA)
SVC Average Accuracy: 95.73% (±0.82%)
KNN Average Accuracy: 94.91% (±0.72%)
Individual SVC scores: ['94.78%', '95.27%', '95.76%', '95.60%', '97.22%']
Individual KNN scores: ['93.96%', '94.62%', '94.62%', '96.08%', '95.26%']


In [ ]:
# Concatenate features from DenseNet121 and VGG16 models along the feature axis
# This creates fused feature vectors by combining both feature sets for each sample
# DenseNet121 features (50176 dim) + VGG16 features (25088 dim) = Fused features (75264 dim)
features = np.concatenate((featuresb1, featuresb2), axis=1)

In [ ]:
featuresf =  features

In [ ]:
labelsf = [int(0) if x=='meningioma' else x for x in labelsf]
labelsf= [int(1) if x=='glioma' else x for x in labelsf]
labelsf= [int(2) if x=='pituitary' else x for x in labelsf]

In [ ]:
# Convert to numpy arrays if they aren't already
featuresf_array = np.array(featuresf)
labelsf_array = np.array(labelsf)

# Initialize classification models with fixed random state for reproducibility
svc = SVC(C=10, random_state=42)
knn = KNeighborsClassifier(n_neighbors=1)
# Initialize 5-fold stratified cross-validation to maintain class distribution
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store accuracy scores for each fold
svc_scores = []
knn_scores = []

print("Starting 5-Fold Cross Validation with fused features...")
print("Configuration: PCA applied after split (1000 components), no SMOTE")

for fold, (train_idx, test_idx) in enumerate(kf.split(featuresf_array, labelsf_array), 1):
    print(f"\n--- Processing Fold {fold}/5 ---")

    # Split data into training and testing sets for current fold
    X_train, X_test = featuresf_array[train_idx], featuresf_array[test_idx]
    y_train, y_test = labelsf_array[train_idx], labelsf_array[test_idx]

    # Apply PCA for dimensionality reduction (75264 -> 1000 features)
    # Important: Fit PCA ONLY on training data to avoid data leakage
    pca = PCA(n_components=1000, random_state=42)
    X_train_pca = pca.fit_transform(X_train)  # Fit and transform training data
    X_test_pca = pca.transform(X_test)        # Transform test data using same PCA model

    # Train and evaluate Support Vector Classifier
    svc.fit(X_train_pca, y_train)
    y_pred_svc = svc.predict(X_test_pca)
    svc_score = accuracy_score(y_test, y_pred_svc) * 100  # Convert to percentage
    svc_scores.append(svc_score)

    # Train and evaluate K-Nearest Neighbors classifier
    knn.fit(X_train_pca, y_train)
    y_pred_knn = knn.predict(X_test_pca)
    knn_score = accuracy_score(y_test, y_pred_knn) * 100  # Convert to percentage
    knn_scores.append(knn_score)

    print(f"Fold {fold} Results:")
    print(f"  SVC Accuracy: {svc_score:.2f}%")
    print(f"  KNN Accuracy: {knn_score:.2f}%")

# Display comprehensive final results
print("\n" + "="*50)
print("FINAL 5-FOLD CROSS VALIDATION RESULTS")
print("Feature Source: Fused DenseNet121 + VGG16 (75264 features reduced to 1000 via PCA)")
print("="*50)
print(f"SVC Average Accuracy: {np.mean(svc_scores):.2f}% (±{np.std(svc_scores):.2f}%)")
print(f"KNN Average Accuracy: {np.mean(knn_scores):.2f}% (±{np.std(knn_scores):.2f}%)")
print(f"Individual SVC scores: {[f'{s:.2f}%' for s in svc_scores]}")
print(f"Individual KNN scores: {[f'{s:.2f}%' for s in knn_scores]}")
print("="*50)

Starting 5-Fold Cross Validation with fused features...
Configuration: PCA applied after split (1000 components), no SMOTE

--- Processing Fold 1/5 ---
Fold 1 Results:
  SVC Accuracy: 96.25%
  KNN Accuracy: 96.25%

--- Processing Fold 2/5 ---
Fold 2 Results:
  SVC Accuracy: 97.06%
  KNN Accuracy: 97.23%

--- Processing Fold 3/5 ---
Fold 3 Results:
  SVC Accuracy: 96.90%
  KNN Accuracy: 96.41%

--- Processing Fold 4/5 ---
Fold 4 Results:
  SVC Accuracy: 96.41%
  KNN Accuracy: 97.72%

--- Processing Fold 5/5 ---
Fold 5 Results:
  SVC Accuracy: 98.53%
  KNN Accuracy: 97.39%

FINAL 5-FOLD CROSS VALIDATION RESULTS
Feature Source: Fused DenseNet121 + VGG16 (75264 features reduced to 1000 via PCA)
SVC Average Accuracy: 97.03% (±0.81%)
KNN Average Accuracy: 97.00% (±0.57%)
Individual SVC scores: ['96.25%', '97.06%', '96.90%', '96.41%', '98.53%']
Individual KNN scores: ['96.25%', '97.23%', '96.41%', '97.72%', '97.39%']


In [ ]:
# Convert to numpy arrays if they aren't already
featuresf_array = np.array(featuresf)
labelsf_array = np.array(labelsf)

# Initialize classification models with fixed random state for reproducibility
svc = SVC(C=10, random_state=42)
knn = KNeighborsClassifier(n_neighbors=1)
# Initialize 5-fold stratified cross-validation to maintain class distribution
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store accuracy scores for each fold
svc_scores = []
knn_scores = []

print("Starting 5-Fold Cross Validation with fused features...")
print("Configuration: Borderline-SMOTE + PCA (1000 components)")

for fold, (train_idx, test_idx) in enumerate(kf.split(featuresf_array, labelsf_array), 1):
    print(f"\n--- Processing Fold {fold}/5 ---")

    # Split data into training and testing sets for current fold
    X_train, X_test = featuresf_array[train_idx], featuresf_array[test_idx]
    y_train, y_test = labelsf_array[train_idx], labelsf_array[test_idx]

    # Apply Borderline-SMOTE ONLY to training data to handle class imbalance
    # Important: SMOTE is applied before PCA and only on training data
    smote = BorderlineSMOTE(sampling_strategy='not majority',
                           kind='borderline-1',
                           k_neighbors=1,
                           m_neighbors=5,
                           random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    # Apply PCA for dimensionality reduction (75264 -> 1000 features)
    # Important: Fit PCA ONLY on SMOTE-augmented training data to avoid data leakage
    pca = PCA(n_components=1000, random_state=42)
    X_train_pca = pca.fit_transform(X_train_smote)  # Fit and transform augmented training data
    X_test_pca = pca.transform(X_test)              # Transform test data using same PCA model

    # Train and evaluate Support Vector Classifier
    svc.fit(X_train_pca, y_train_smote)
    y_pred_svc = svc.predict(X_test_pca)
    svc_score = accuracy_score(y_test, y_pred_svc) * 100  # Convert to percentage
    svc_scores.append(svc_score)

    # Train and evaluate K-Nearest Neighbors classifier
    knn.fit(X_train_pca, y_train_smote)
    y_pred_knn = knn.predict(X_test_pca)
    knn_score = accuracy_score(y_test, y_pred_knn) * 100  # Convert to percentage
    knn_scores.append(knn_score)

    print(f"Fold {fold} Results:")
    print(f"  SVC Accuracy: {svc_score:.2f}%")
    print(f"  KNN Accuracy: {knn_score:.2f}%")

# Display comprehensive final results
print("\n" + "="*50)
print("FINAL 5-FOLD CROSS VALIDATION RESULTS")
print("Feature Source: Fused DenseNet121 + VGG16 (75264 features)")
print("Preprocessing: Borderline-SMOTE + PCA (1000 components)")
print("="*50)
print(f"SVC Average Accuracy: {np.mean(svc_scores):.2f}% (±{np.std(svc_scores):.2f}%)")
print(f"KNN Average Accuracy: {np.mean(knn_scores):.2f}% (±{np.std(knn_scores):.2f}%)")
print(f"Individual SVC scores: {[f'{s:.2f}%' for s in svc_scores]}")
print(f"Individual KNN scores: {[f'{s:.2f}%' for s in knn_scores]}")
print("="*50)

Starting 5-Fold Cross Validation with fused features...
Configuration: Borderline-SMOTE + PCA (1000 components)

--- Processing Fold 1/5 ---
Fold 1 Results:
  SVC Accuracy: 96.08%
  KNN Accuracy: 95.43%

--- Processing Fold 2/5 ---
Fold 2 Results:
  SVC Accuracy: 97.06%
  KNN Accuracy: 97.55%

--- Processing Fold 3/5 ---
Fold 3 Results:
  SVC Accuracy: 96.90%
  KNN Accuracy: 96.08%

--- Processing Fold 4/5 ---
Fold 4 Results:
  SVC Accuracy: 96.41%
  KNN Accuracy: 97.72%

--- Processing Fold 5/5 ---
Fold 5 Results:
  SVC Accuracy: 98.69%
  KNN Accuracy: 97.06%

FINAL 5-FOLD CROSS VALIDATION RESULTS
Feature Source: Fused DenseNet121 + VGG16 (75264 features)
Preprocessing: Borderline-SMOTE + PCA (1000 components)
SVC Average Accuracy: 97.03% (±0.90%)
KNN Average Accuracy: 96.77% (±0.88%)
Individual SVC scores: ['96.08%', '97.06%', '96.90%', '96.41%', '98.69%']
Individual KNN scores: ['95.43%', '97.55%', '96.08%', '97.72%', '97.06%']
